In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
# d = [[1, 2], [1, 2]]
# n = 2   # 两个设施点
# m = 2   # 两个需求点
d = [[1, 2], [3, 4], [5, 6]]
n = 2   # 两个设施点
m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [5]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j - 1] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [6]:
def gnrt_Hd(n, m):
  mn = m * n
  Hd = np.zeros((2**mn, 2**mn)).astype(np.complex128)
  for i in range(mn):
    j = (i + 1) % mn
    Hd += (add_in_target(mn, i, GateX) @ add_in_target(mn, j, GateX) + add_in_target(mn, i, GateY) @ add_in_target(mn, j, GateY))
  return -Hd


In [7]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n, m)), range(m * n , 2 * m * n))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [8]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌───────────┐┌─────────┐ ┌───────────┐┌─────────┐ ┌───────────┐»
    q_0: ┤ H ├┤0          ├┤ Rx(π/4) ├─┤0          ├┤ Rx(π/4) ├─┤0          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_1: ┤ H ├┤1          ├┤ Rx(π/4) ├─┤1          ├┤ Rx(π/4) ├─┤1          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_2: ┤ H ├┤2          ├┤ Rx(π/4) ├─┤2          ├┤ Rx(π/4) ├─┤2          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_3: ┤ H ├┤3          ├┤ Rx(π/4) ├─┤3          ├┤ Rx(π/4) ├─┤3          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_4: ┤ H ├┤4          ├┤ Rx(π/4) ├─┤4          ├┤ Rx(π/4) ├─┤4          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_5: ┤ H ├┤5          ├┤ Rx(π/4) ├─┤5          ├┤ Rx(π/4) ├─┤5          ├»
         ├───┤│           │├─────────┴┐│           │

In [9]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [10]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [11]:
test(40, 1, np.full(1 * 2, np.pi))

Iteration 10, Result: 148.296
Iteration 20, Result: 137.8845
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 137.8715
       x: [ 4.448e+00  4.168e+00]
    nfev: 29
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
11010101000000 2.2% 135
11100101000000 1.4% 134
11001101000000 1.4% 136
11010011000000 1.4% 136
11010110000000 1.4% 134
11011001000000 1.3% 134
11110100000000 1.3% 130
11010101001000 1.0% 95
11010101100000 1.0% 95
11010101000010 0.9% 95
01010101000000 0.9% 13
11101010000000 0.8% 132
11001110000000 0.8% 135
11111000000000 0.8% 129
11100011000000 0.7% 135
11001101000010 0.6% 96
11010110000010 0.6% 174
11110100000010 0.6% 90
11010110001000 0.6% 94
01010011000000 0.6% 94


In [12]:
test(40, 2, np.full(2 * 2, np.pi))

Iteration 10, Result: 224.469
Iteration 20, Result: 160.145
Iteration 30, Result: 164.912
Iteration 40, Result: 160.68
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 160.2735
       x: [ 4.368e+00  3.095e+00  4.100e+00  3.126e+00]
    nfev: 45
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
00101010000000 1.9% 129
00110100000000 1.6% 127
00011001000000 1.5% 131
00010110000000 1.5% 131
00010011000000 1.5% 133
00001101000000 1.5% 133
00100101000000 1.5% 131
00111000000000 1.4% 126
00001110000000 1.4% 132
00100011000000 1.3% 132
00010101000000 1.2% 132
01101010000000 0.9% 250
00110010000000 0.8% 128
00001011000000 0.7% 134
00100110000000 0.7% 130
00101001000000 0.7% 130
00011010000000 0.7% 130
00101100000000 0.7% 128
01101010010000 0.7% 210
01101010000100 0.7% 210


In [13]:
test(40, 3, np.full(3 * 2, np.pi))

Iteration 10, Result: 266.3955
Iteration 20, Result: 272.4
Iteration 30, Result: 208.5955
Iteration 40, Result: 142.095
Iteration 50, Result: 120.667
Iteration 60, Result: 114.0765
Iteration 70, Result: 113.608
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 114.7505
       x: [ 4.656e+00  2.438e+00  3.388e+00  4.797e+00  3.148e+00
            2.897e+00]
    nfev: 74
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
10101010000000 9.3% 11
10111000000000 4.8% 88
10001110000000 4.0% 94
10100011000000 3.9% 94
10101100000000 2.4% 90
10101010000010 2.3% 51
10110010000000 2.3% 90
10101010001000 2.0% 51
10000111000000 1.5% 177
10101010100000 1.4% 51
00111000000000 1.4% 126
00001110000000 1.3% 132
10101001000000 1.3% 92
10011010000000 1.3% 92
00100011000000 1.2% 132
00001011000000 1.1% 134
10111000000010 1.0% 48
10110001000000 1.0% 171
10001110000

In [14]:
test(40, 5, np.full(5 * 2, np.pi/4))

Iteration 10, Result: 201.6845
Iteration 20, Result: 237.7125
Iteration 30, Result: 232.151
Iteration 40, Result: 211.259
Iteration 50, Result: 174.7075
Iteration 60, Result: 184.8215
Iteration 70, Result: 169.1025
Iteration 80, Result: 166.8515
Iteration 90, Result: 167.3345
Iteration 100, Result: 161.777
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 163.858
       x: [ 8.034e-01  1.784e+00  1.767e+00  7.764e-01  7.800e-01
            7.861e-01  7.727e-01  1.787e+00  7.821e-01  1.904e+00]
    nfev: 106
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
01100110000000 1.0% 171
01011001000000 0.9% 92
00000111000000 0.8% 135
01010101000000 0.7% 13
10100011000000 0.6% 94
00101010000000 0.6% 129
00010011000000 0.5% 133
10101010000000 0.5% 11
11001101100000 0.5% 96
01100011000000 0.5% 173
10001011000000 0.5% 96
10001110000000 0.5% 94
010011100

In [15]:
# for i in range(1, 20):
#   print(f'depth == {i}')
#   test(20, i, np.ones(2 * i))